### This file is used for finding authors' information on linux kernel.
### Educational purpose only, do not use for other purpose please!
#### Author: Yichen Wang, Lanzhou University   Sid:320180940341

In [2]:
import unicodedata
import csv
from subprocess import Popen, PIPE

We recognise every author as an object

In [3]:
class author:
    def __init__(self,name):
        """
        For each author you need to provide his or her name
        Key arguments:
        name -- author's name
        commit -- each commits' time
        """
        self.name = name
        self.commit = []
     
    # add_time is to add commit's time to the commit
    def add_time(self,time):
        self.commit.append(time)
       
    # get_times is to get total times of commits
    def get_times(self):
        return len(self.commit)
    
    # get_average is to get the average time difference between time
    def get_average(self):
        if len(self.commit) == 1:
            return 0
        sum = 0
        for i in range(0,len(self.commit)-1):
            self.commit[i] = abs(self.commit[i+1]-self.commit[i])
            sum += self.commit[i]
        return sum//(len(self.commit)-1)

We recognise every log as an object

In [4]:
class log:
    def __init__(self,verran):
        """
        For each log you need to provide the range in a certain version you need to check
        Key arguments:
        verran -- the range in a certain version
        """
        self.verran = verran
        self.repo = "D:/kernel/linux-stable"
        self.author = {}
    
    # get_data is to get the origin data that we need to work on 
    def get_data(self):
        cmd = ["git", "log", "-P", "--no-merges", "--pretty=format:\"  %an : %ct\"", self.verran]
        p = Popen(cmd, cwd=self.repo, stdout=PIPE)
        data, res = p.communicate()
        data = unicodedata.normalize(u'NFKD', data.decode(encoding="utf-8", errors="ignore"))
        return data
    
    # get_author is to return two list which store the info
    def get_author(self):
        au = []
        date = []
        for line in self.get_data().split("\n"):
            for i in line.split(":"):
                if i[3].isdigit():
                    date.append(eval(i[1:11]))
                else:
                    au.append(i[3:])
        return au,date
    
    # complete_author is to fill in the dict defined in __init__
    def complete_author(self):
        au,date = self.get_author()
        for i in range(0,len(au)):
            if au[i] in self.author:
                self.author[au[i]].append(date[i])
            else:
                self.author[au[i]] = [date[i]]
        return self.author
        

Then we change the dict to the class

In [5]:
def dict2class(d={}):
    auth = []
    for i in d.keys():
        new = author(i)
        for j in d[i]:
            new.add_time(j)
        auth.append(new)
    return auth

Main

In [11]:
def main():
    b = log("v4.1..v4.9").complete_author()
    au = dict2class(b)
    info = []
    with open('result.csv','w',encoding='utf-8') as csvfile:
        spamwriter=csv.writer(csvfile, dialect='excel')
        spamwriter.writerow(["author","times","average"])
        for i in au:
            info = [i.name, i.get_times(), i.get_average()]
            spamwriter.writerow(info)

if __name__ == "__main__":
    main() 